In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

In [2]:
os.environ["LANGCHAIN_PROJECT"]= "MIRA-local-dev"
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

In [3]:
MONGODB_ATLAS_CLUSTER_URI = os.getenv('MONGODB_ATLAS_CLUSTER_URI')
DB_NAME = os.getenv('DB_NAME')
COLLECTION_NAME = os.getenv('COLLECTION_NAME')
ATLAS_VECTOR_SEARCH_INDEX_NAME = os.getenv('ATLAS_VECTOR_SEARCH_INDEX_NAME')

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_mongodb.vectorstores import MongoDBAtlasVectorSearch

vector_search = MongoDBAtlasVectorSearch.from_connection_string(
        MONGODB_ATLAS_CLUSTER_URI,
        f"{DB_NAME}.{COLLECTION_NAME}",
        OpenAIEmbeddings(),
        index_name= ATLAS_VECTOR_SEARCH_INDEX_NAME
    )

In [5]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
retriever = vector_search.as_retriever(search_kwargs={"k":5})

In [6]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [7]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [10]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "What is bipolar disorder?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])


print(ai_msg_1["answer"])

Bipolar disorder is a mental illness characterized by dramatic shifts in mood, considered a mood disorder. People with bipolar disorder experience high and low moods known as mania or hypomania and depression. It affects about 2.8% of the U.S. population, with various types such as Bipolar I, Bipolar II, cyclothymia, and unspecified bipolar disorder.


In [11]:
chat_history

[HumanMessage(content='What is bipolar disorder?'),
 'Bipolar disorder is a mental illness characterized by dramatic shifts in mood, considered a mood disorder. People with bipolar disorder experience high and low moods known as mania or hypomania and depression. It affects about 2.8% of the U.S. population, with various types such as Bipolar I, Bipolar II, cyclothymia, and unspecified bipolar disorder.']

In [12]:
second_question = "how to overcome this?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

In [14]:
print(ai_msg_2["answer"])

Treatment for bipolar disorder typically involves a combination of medication like mood stabilizers and psychotherapy. Stress management strategies, early symptom recognition, and maintaining a healthy lifestyle with proper nutrition, exercise, and sleep patterns are also important. Complementary approaches such as meditation and spirituality can help regulate mood and manage symptoms effectively.


In [16]:
chat_history.extend([HumanMessage(content=second_question), ai_msg_2["answer"]])

In [17]:
chat_history

[HumanMessage(content='What is bipolar disorder?'),
 'Bipolar disorder is a mental illness characterized by dramatic shifts in mood, considered a mood disorder. People with bipolar disorder experience high and low moods known as mania or hypomania and depression. It affects about 2.8% of the U.S. population, with various types such as Bipolar I, Bipolar II, cyclothymia, and unspecified bipolar disorder.',
 HumanMessage(content='how to overcome this?'),
 'Treatment for bipolar disorder typically involves a combination of medication like mood stabilizers and psychotherapy. Stress management strategies, early symptom recognition, and maintaining a healthy lifestyle with proper nutrition, exercise, and sleep patterns are also important. Complementary approaches such as meditation and spirituality can help regulate mood and manage symptoms effectively.']

In [18]:
third_question = "provide some more solutions"
ai_msg_3 = rag_chain.invoke({"input": third_question, "chat_history": chat_history})
print(ai_msg_3["answer"])

In addition to medication and psychotherapy, stress management techniques like exercise, meditation, and maintaining a regular schedule can be beneficial for managing bipolar disorder. Complementary health approaches such as spirituality and appropriate nutrition can also support mood regulation. Early identification of symptoms and seeking help from mental health professionals are crucial steps in effectively managing the condition.


In [21]:
chat_history

[HumanMessage(content='What is bipolar disorder?'),
 'Bipolar disorder is a mental illness characterized by dramatic shifts in mood, considered a mood disorder. People with bipolar disorder experience high and low moods known as mania or hypomania and depression. It affects about 2.8% of the U.S. population, with various types such as Bipolar I, Bipolar II, cyclothymia, and unspecified bipolar disorder.',
 HumanMessage(content='how to overcome this?'),
 'Treatment for bipolar disorder typically involves a combination of medication like mood stabilizers and psychotherapy. Stress management strategies, early symptom recognition, and maintaining a healthy lifestyle with proper nutrition, exercise, and sleep patterns are also important. Complementary approaches such as meditation and spirituality can help regulate mood and manage symptoms effectively.',
 HumanMessage(content='how to overcome this?')]

In [22]:
chat_history.extend([HumanMessage(content=third_question), ai_msg_3["answer"]])

In [25]:
chat_history

[HumanMessage(content='What is bipolar disorder?'),
 'Bipolar disorder is a mental illness characterized by dramatic shifts in mood, considered a mood disorder. People with bipolar disorder experience high and low moods known as mania or hypomania and depression. It affects about 2.8% of the U.S. population, with various types such as Bipolar I, Bipolar II, cyclothymia, and unspecified bipolar disorder.',
 HumanMessage(content='how to overcome this?'),
 'Treatment for bipolar disorder typically involves a combination of medication like mood stabilizers and psychotherapy. Stress management strategies, early symptom recognition, and maintaining a healthy lifestyle with proper nutrition, exercise, and sleep patterns are also important. Complementary approaches such as meditation and spirituality can help regulate mood and manage symptoms effectively.',
 HumanMessage(content='provide some more solutions'),
 'In addition to medication and psychotherapy, stress management techniques like exe